In [64]:
depth_tree = 7

In [65]:
""" Import libraries """
import json
import pandas as pd
import psycopg2
import nbimporter
from Core import json_numpy_serializer

pd.set_option('display.float_format', lambda x: '%.6f' % x)
pd.set_option('display.max_rows', None)

In [66]:
""" Read parameters """
with open("para.json", "r") as json_file:
    para = json.load(json_file)
fold_para = para["fold_para"]
with open("{}/para_{}.json".format(fold_para, depth_tree), "r") as json_file:
    para = json.load(json_file)

In [67]:
""" Database Connection """
conn_params = "dbname='trajectory' user='postgres' host='localhost' password='990721' port=5432"
conn = psycopg2.connect(conn_params)
cur = conn.cursor()

In [68]:
""" Query data distribution """
query = """
SELECT indexing_key, sum(pg_column_size(geometry)) AS size
FROM traj_{}
WHERE geometry is not null
GROUP BY indexing_key;
""".format(depth_tree)

cur.execute(query)
records = cur.fetchall()
columns = [desc[0] for desc in cur.description]
df = pd.DataFrame(records, columns=columns)

In [69]:
""" Disconnect database """
cur.close()
conn.close()

In [70]:
""" Aggregate data distribution """
dfs = []
for i in range(depth_tree + 1):
    dfs.append(df)
    df = df.copy()
    df["indexing_key"] = df.apply(lambda x: int(x["indexing_key"] / 8), axis=1)
    df = df.groupby('indexing_key')['size'].sum()
    df = df.reset_index()
    df.columns = ['indexing_key', 'size']
dfs.reverse()

In [71]:
""" Transform to list """
dist = []
for i, df in enumerate(dfs):
    dist.append(df.values.tolist())

In [72]:
""" Write parameters """
para["dist"] = dist
with open("{}/para_{}.json".format(fold_para, depth_tree), "w") as json_file:
    json.dump(para, json_file, default=json_numpy_serializer)